In [1]:
import pickle
import pandas as pd
import os

import torch
with torch.profiler.profile() as profiler:
        pass


import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
torch.__version__

'1.12.1'

In [5]:
class DogInput:
    def __init__(self, dogid, raceid,stats, dog,dog_box, hidden_state, bfsp, sp) -> None:
        self.dogid= dogid
        self.raceid = raceid
        self.stats = stats.to('cuda:0')
        self.dog = dog
        self.gru_cell = hidden_state.float().to('cuda:0')
        self.visited = 0
        self.bfsp = bfsp
        self.gru_cell_out = None

        
        
    def lstm_i(self, hidden_state):
        self.gru_cell = hidden_state
        # self.lstmCellh=self.lstmCellh.to(device)
        # self.lstmCellc=self.lstmCellc.to(device)
        self.visited = self.visited + 1
        # if self.visited>1:
        #     print("FOUND LEAK")
        #     sasdfasd

    def nextrace(self, raceid):
        self.nextrace_id = raceid

    def prevrace(self, raceid):
        self.prevrace_id = raceid

    def lstm_o(self, lstm_o):
        # print(lstm_o[0]._version)
        hidden_state = lstm_o
        self.gru_cell_out = lstm_o
        if self.nextrace_id==-1:
            pass
        else:
            self.dog.races[self.nextrace_id].lstm_i(hidden_state) #((lh.detach(), lc.detach())) #DETACH

    def detach_state(self):
        self.gru_cell = self.gru_cell.detach()
            


In [6]:
class Dog:
    def __init__(self, dogid,dog_name, hidden_size, layers) -> None:
        self.dogid = dogid
        self.dog_name = dog_name
        # self.raceids = raceids #possible dictionary of race id keys dog stat outs
        self.lstmcell = 0
        self.layers = layers
        self.hidden_size = hidden_size
        self.l_debug = None
        self.races = {}

    def add_races(self, raceid, racedate, stats,nextraceid, prevraceid, box, bfsp=None, sp=None):
        self.races[raceid] = DogInput(self.dogid, raceid, stats, self, box, torch.randn(self.hidden_size), bfsp, sp) #this is the change
        self.races[raceid].nextrace(nextraceid)
        self.races[raceid].prevrace(prevraceid)

In [7]:
class Race:
    def __init__(self, raceid,trackOHE, dist, classes=None):
        self.raceid = raceid
        self.race_dist = dist.to('cuda:0')
        self.race_track = trackOHE.to('cuda:0')
        self.track_name = None
        if classes!=None:
            self.classes =  classes.to('cuda:0')

    def add_dogs(self, dogs_list:DogInput):
        self.dog1 = dogs_list[0]
        self.dog2 = dogs_list[1]
        self.dog3 = dogs_list[2]
        self.dog4 = dogs_list[3]
        self.dog5 = dogs_list[4]
        self.dog6 = dogs_list[5]
        self.dog7 = dogs_list[6]
        self.dog8 = dogs_list[7]
        self.dogs = dogs_list

    def add_track_name(self, track_name):
        self.track_name = track_name


    def nn_input(self):
        input = torch.cat([x.stats for x in self.dogs], dim = 0)
        full_input = torch.cat((self.race_dist,self.race_track, input), dim=0).to(device='cuda:0')
        self.full_input = full_input
        return full_input

    def lstm_input(self, pred=False):
        if pred:
            print('pred')
        else:
            l_input = [x.gru_cell for x in self.dogs]
        return l_input

    def lstm_detach(self):
        [x.detach_state for x in self.dogs]

    def list_dogs(self):
        dogs_l = [x for x in self.dogs]
        return dogs_l

    def pass_gru_output(self, hidden_states):
        for i,dog in enumerate(self.dogs):
            hs = hidden_states[i]
            #hs = hs.detach()
            dog.lstm_o(hs) #.clone())

In [8]:
class Races:
    def __init__(self, hidden_size, layers, batch_size = 100) -> None:
        self.racesDict = {}
        self.dogsDict = {}
        self.raceIDs = []
        self.hidden_size = hidden_size
        self.layers = layers
        self.getter = operator.itemgetter(*range(batch_size))

    def add_race(self,raceid:str, trackOHE, dist, classes=None):
        self.racesDict[raceid] = Race(raceid, trackOHE, dist, classes)
        self.raceIDs.append(raceid)

    def add_dog(self,dogid, dog_name):
        if dogid not in self.dogsDict.keys():
            self.dogsDict[dogid] = Dog(dogid,dog_name, self.hidden_size, self.layers)
        # else:
        #     self.dogsDict[dogid] = self.dogsDict[dogid]

    def get_race_input(self, idx) -> Race:
        if len(idx)==1:
            race = self.racesDict[self.raceIDs[idx]]
            print(f"returing race {race}")
            return race
        else:
            raceidx = operator.itemgetter(*idx)
            race_batch_id = raceidx(self.raceIDs)
            races = [self.racesDict[x] for x in race_batch_id] #Returns list of class Race
        
        return races #self.racesDict[raceidx]

    def get_race_classes(self, idx):
        raceidx = self.raceIDs[idx]
        classes = [x for x in self.raceDict[raceidx].classes]
        return classes

    def get_race_weights(self, races):
        print("weights")

    def reset_all_lstm_states(self):
        for race in self.racesDict.values:
            for dog in race.dogs:
                dog.lstmCellc = torch.rand(self.hidden_size)
                dog.lstmCellh = torch.rand(self.hidden_size)
                dog.gru_cell = torch.rand(self.hidden_size)

    def detach_all_hidden_states(self):
        for race in self.racesDict.values():
            for dog in race.dogs:
                dog.gru_cell = dog.gru_cell.detach()

    def detach_given_hidden_states(self, races):
        for race in races:
            for dog in race.dogs:
                dog.gru_cell = dog.gru_cell.detach()

    def create_hidden_states_dict(self):
        self.hidden_states_dict = {}
        for race in self.racesDict.values():
            race_id = race.raceid
            for dog in race.dogs:
                dog_id = dog.dogid
                key = race_id+'_'+dog_id
                val = dog.gru_cell_out
                self.hidden_states_dict[key] = val

    def fill_hidden_states_from_dict(self, hidden_dict):
        for race in self.racesDict.values():
            race_id = race.raceid
            for dog in race.dogs:
                dog_id = dog.dogid
                dog_prev_race_id = dog.prevrace_id
                key = str(dog_prev_race_id)+'_'+dog_id
                try:
                    val = hidden_dict[key]
                    if val != None:
                        dog.gru_cell = val
                    else:
                        dog.gru_cell = torch.rand(self.hidden_size)
                except KeyError as e:
                    print(f'Key error {e}')
                    val = torch.rand(self.hidden_size)
                    dog.gru_cell = val
                    print(f"race in = {dog.gru_cell}")
                print(key,val)

    def to_cuda(self):
        for race in self.racesDict.values():
            race_id = race.raceid
            for dog in race.dogs:
                dog.gru_cell = dog.gru_cell.to(device)


In [9]:
class GRUNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.3):
        super(GRUNet, self).__init__()
        self.gru1 = nn.GRUCell(input_size, hidden_size)
        self.gru2 = nn.GRUCell(input_size, hidden_size)
        self.gru3 = nn.GRUCell(input_size, hidden_size)
        self.gru4 = nn.GRUCell(input_size, hidden_size)
        self.gru5 = nn.GRUCell(input_size, hidden_size)
        self.gru6 = nn.GRUCell(input_size, hidden_size)
        self.gru7 = nn.GRUCell(input_size, hidden_size)
        self.gru8 = nn.GRUCell(input_size, hidden_size)
        self.rl1 = nn.ReLU()
        self.drop1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size * 8, 64)
        self.drop2 = nn.Dropout(dropout)
        self.rl2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 8)
        self.hidden_size = hidden_size

    # x represents our data
    def forward(self, race: Race):
        x = torch.stack([r.full_input.float() for r in race])#.detach()

        # creates list of LSTM data
        hidden_state_in = [list(i) for i in zip(*[r.lstm_input() for r in race])]

        # creates list of tensors for lstm Cells
        hCell = [torch.stack([x for x in y]) for y in hidden_state_in]
        # print(f"{hCell=}")
        # for h in hCell:
        #     print(h._version)
        #     if h.grad_fn != None:
        #         print(h.grad_fn._saved_self)
        #         print(h.grad_fn._saved_other)
        # for h in hCell:
        #     print(h)

        h1 = self.gru1(x, hCell[0])
        h2 = self.gru2(x, hCell[1])
        h3 = self.gru3(x, hCell[2])
        h4 = self.gru4(x, hCell[3])
        h5 = self.gru5(x, hCell[4])
        h6 = self.gru6(x, hCell[5])
        h7 = self.gru7(x, hCell[6])
        h8 = self.gru8(x, hCell[7])

        lstm_list = [
            h1,
            h2,
            h3,
            h4,
            h5,
            h6,
            h7,
            h8,
        ]

        hCello = [i for i in zip(*[x for x in lstm_list])]
        # cCello = [i for i in zip(*[x[1] for x in lstm_list])]

        for i, r in enumerate(race):
            r.pass_gru_output(hCello[i])
            # r.lstm_detach()
        xhh = torch.cat((h1, h2, h3, h4, h5, h6, h7, h8), dim=1)
        xr1 = self.rl1(xhh)
        xd1 = self.drop1(xr1)
        xh = self.fc2(xd1)
        xd2 = self.drop2(xh)
        xr2 = self.rl2(xd2)
        xf = self.fc3(xr2)

        #output = F.softmax(xf, dim=1)
        #output = nn.LogSoftmax(xf)
        output = xf
        return output


In [10]:
def build_pred_dataset(data, hidden_size):

    #Load in pickeled dataframe
    resultsdf = pickle.load(data)
    dog_stats_df = pd.DataFrame(resultsdf)
    dog_stats_df = dog_stats_df.fillna(-1).drop_duplicates(subset=['dogid', 'raceid'])
    dog_stats_df['stats_cuda'] = dog_stats_df.apply(lambda x: torch.tensor(x['stats']), axis =1)
    dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])

    #Created RaceDB
    raceDB = Races(hidden_size, 1)

    #Fill in dog portion:

    dog_stats_group = dog_stats_df.sort_values(['date']).groupby(["dogid"])

    for i,j in tqdm(dog_stats_group):
        raceDB.add_dog(i, j.dog_name.iloc[0])
        j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),-1, x['prev_race'], x['box']), axis=1)

    #Fill in races portion
    softmin = nn.Softmin(dim=0)
    races_group = dog_stats_df.groupby(['raceid'])

    null_dog = Dog("nullDog", "no_name", raceDB.hidden_size, raceDB.layers)
    null_dog_i = DogInput("nullDog", "-1", torch.zeros(16), null_dog,0, torch.zeros(raceDB.hidden_size),0,0)
    null_dog_i.nextrace(-1)
    null_dog_i.prevrace(-1)

    null_dog_list = [null_dog] * 8
    #TO FIX LATER PROPER BOX PLACEMENT #FIXED

    races_group = dog_stats_df.groupby(['raceid'])
    for i,j in tqdm(races_group):
    #Track info tensors
        dist = torch.tensor([j.dist.iloc[0]]) 
        trackOHE = torch.tensor(j.trackOHE.iloc[0])

        empty_dog_list = [null_dog_i]*8
        boxes_list = [x for x in j['box']]      
        dog_list = [raceDB.dogsDict[x].races[i] for x in j["dogid"]]

        for n,x in enumerate(boxes_list):
            empty_dog_list[x-1] = dog_list[n]
        
        raceDB.add_race(i,trackOHE,dist)
        
        # List of Dog Input??
        raceDB.racesDict[i].add_dogs(empty_dog_list)
        raceDB.racesDict[i].nn_input()
        raceDB.racesDict[i].add_track_name(j.track_name.iloc[0])
        raceDB.racesDict[i].track_name = j.track_name.iloc[0]
        raceDB.racesDict[i].grade = j.race_grade.iloc[0]

    print(f"number of races = {len(raceDB.racesDict)}, number of unique dogs = {len(raceDB.dogsDict)}")
    return raceDB



In [11]:
def build_dataset(data, hidden_size):

    #Load in pickeled dataframe
    resultsdf = pickle.load(data)
    dog_stats_df = pd.DataFrame(resultsdf)
    dog_stats_df = dog_stats_df.fillna(-1).drop_duplicates(subset=['dogid', 'raceid'])
    dog_stats_df['stats_cuda'] = dog_stats_df.apply(lambda x: torch.tensor(x['stats']), axis =1)
    dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])

    #Created RaceDB
    raceDB = Races(hidden_size, 1)

    num_features_per_dog = len(dog_stats_df['stats'][0])
    print(f"{num_features_per_dog=}")

    #Fill in dog portion:

    dog_stats_group = dog_stats_df.sort_values(['date']).groupby(["dogid"]) 

    for i,j in tqdm(dog_stats_group):
        j["next_race"] = j["raceid"].shift(-1).fillna(-1)
        j["prev_race"] = j["raceid"].shift(1).fillna(-1)
        raceDB.add_dog(i, j.dog_name.iloc[0])
        j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),x['next_race'], x['prev_race'], x['box'], x['bfSP'], x['startprice']), axis=1)

    #Fill in races portion
    softmin = nn.Softmin(dim=0)
    races_group = dog_stats_df.groupby(['raceid'])

    null_dog = Dog("nullDog", "no_name", raceDB.hidden_size, raceDB.layers)
    null_dog_i = DogInput("nullDog", "-1", torch.zeros(num_features_per_dog), null_dog,0, torch.zeros(raceDB.hidden_size),0,0)
    null_dog_i.nextrace(-1)
    null_dog_i.prevrace(-1)

    null_dog_list = [null_dog] * 8
    #TO FIX LATER PROPER BOX PLACEMENT #FIXED
    dog_stats_df = dog_stats_df.sort_values('date')
    races_group = dog_stats_df.groupby(['raceid'])
    for i,j in tqdm(races_group):
    #Track info tensors
        dist = torch.tensor([j.dist.iloc[0]]) 
        trackOHE = torch.tensor(j.trackOHE.iloc[0])
        #margins
        empty_dog_list = [null_dog_i]*8
        empty_margin_list = [20]*8
        empty_place_list = [8]*8

        places_list = [x for x in j["place"]]
        boxes_list = [x for x in j['box']]
        margin_list = [x for x in j["margin"]]
        
        dog_list = [raceDB.dogsDict[x].races[i] for x in j["dogid"]]

        #adjustedMargin = [margin_list[x-1] for x in boxes_list]
        for n,x in enumerate(boxes_list):
            empty_margin_list[x-1] = margin_list[n]
            empty_dog_list[x-1] = dog_list[n]
            empty_place_list[x-1] = places_list[n]
        adjustedMargin = softmin(torch.tensor(empty_margin_list)).to('cuda:0')
        # adjustedMargin = softmin(torch.tensor(empty_place_list)).to('cuda:0')
        # adjustedMargin = torch.tensor(empty_place_list).to('cuda:0')
        #adjusted_dog_list = [dog_list[x-1] for x in boxes_list]
        
        raceDB.add_race(i,trackOHE,dist, adjustedMargin)
        
        
        # List of Dog Input??
        raceDB.racesDict[i].add_dogs(empty_dog_list)
        raceDB.racesDict[i].nn_input()
        raceDB.racesDict[i].track_name = j.track_name.iloc[0]
        raceDB.racesDict[i].grade = j.race_grade.iloc[0]

    print(f"number of races = {len(raceDB.racesDict)}, number of unique dogs = {len(raceDB.dogsDict)}")
    return raceDB


In [12]:
def model_saver(model, optimizer, epoch, loss, hidden_state_dict):
    
    pathtofolder = "C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model"
    model_name = wandb.run.name
    isExist = os.path.exists(
        f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
    )
    if isExist:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )
    else:
        print("created path")
        os.makedirs(
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
        )
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": loss,
                "db":hidden_state_dict,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )

In [13]:
#Testing
def validate_model(model,raceDB,criterion, batch_size, example_ct, epoch_loss, batch_ct):
    torch.autograd.set_detect_anomaly(True)
    batch_size=100
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    actuals = []
    preds = []
    grades = []
    tracks = []
    pred_confs = []
    bfsps = []
    start_prices = []
    with torch.no_grad():
        for i in trange(0,123_000,batch_size):
            if ((i+1)%1_000<800):
                continue
            races_idx = range(last,last+batch_size)
            last = i
            race = raceDB.get_race_input(races_idx)
            #tracks.extend([r.track for r in race])
            X = race
            y = torch.stack([x.classes for x in race])
            output = model(X)
            #print(y)

            _, actual = torch.max(y.data, 1)
            conf, predicted = torch.max(output.data, 1)
            correct += (predicted == actual).sum().item()
            
            total += batch_size
            actuals.extend(actual.tolist())
            preds.extend(predicted.tolist())
            pred_confs.extend(conf.tolist())
            tracks.extend([r.track_name for r in race])
            grades.extend([r.grade for r in race])
            for i,dog_idx in enumerate(actual.tolist()):
                bfsps.append(race[i].dogs[dog_idx].bfsp)
                #start_prices.append(race[i].dogs[dog_idx].sp)

            
            loss = criterion(output, y)
            loss_val += loss

    logdf = pd.DataFrame(data = {"actuals":actuals, "preds":preds,"conf":pred_confs, "grade":grades, "track":tracks, "bfsps":bfsps})#, "sp":start_prices })

    table = wandb.Table(dataframe=logdf)
    try:
        wandb.log({"table_key": table})
    except Exception as e:
        print("e")
    print(f"accuray: {correct/total}")
    wandb.log({"accuracy": correct/total, "loss_val": loss_val/(50_000/batch_size), "correct": correct,"epoch_loss": epoch_loss/batch_ct})

    return correct/total

In [14]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)

In [55]:
def train(model, raceDB:Races, criterion, optimizer,scheduler, config=None):
    torch.autograd.set_detect_anomaly(True)

    last = 0
    batch_size = 8
    example_ct = 0  # number of examples seen
    batch_ct = 0
    m = nn.LogSoftmax(dim=1)

    losses = []
    for epoch in range(2_000):
        model.train()
        #epoch_loss = torch.Tensor(0).to(device).requires_grad_(True)
        batch_ct = 0

        setup_loss = 1
        
        for i in range(0,123_000,batch_size):
            last = i
            if last>123_000:
                break
            if ((last+1)%1_000>=800):
                if setup_loss:
                    continue
                else:
                    print()
                    print(f"Before Loss last {last}")
                    print(f"final epoch before loss = {epoch_loss}")
                    optimizer.zero_grad()
                    epoch_loss.backward()
                    optimizer.step()
                    model.zero_grad()
                    #losses.append(epoch_loss.item())
                    raceDB.detach_all_hidden_states()
                    setup_loss = 1
                    wandb.log({"loss": epoch_loss}, step = example_ct)
                    last = i+200
                    #i = last
                    print(f"After Loss last {last}")

                    # print("in validation area")
                    # print(f"{i=}")
                    continue
            batch_ct += 1
            # print("regular")
            races_idx = range(last,last+batch_size)
            
            race = raceDB.get_race_input(races_idx)
            #race = [raceDB.racesDict[r],raceDB.racesDict[r]]
            X = race
            # print(f"race id = {X[0].raceid}")

            y = torch.stack([x.classes for x in race])
            _, actual = torch.max(y.data, 1)
            output = model(X)

            
            #print(output,y)
            example_ct +=  batch_size

            loss = criterion(output, y)# *weights
            if setup_loss:
                epoch_loss = loss
                setup_loss=0
            #oss = criterion(m(output), torch.flatten(actual))
            
            #loss.backward(retain_graph=True)  
            #loss.backward()
            #optimizer.step()
            if ((batch_ct + 1) % 25) == 0:                    
                    train_log(loss, example_ct, epoch)

            epoch_loss = epoch_loss + loss
            #output = model(X)

            if ((batch_ct + 1) % 10) == 0:
                continue
                print(f"final epoch before loss = {epoch_loss}")
                optimizer.zero_grad()
                epoch_loss.backward()
                optimizer.step()
                model.zero_grad()
                losses.append(epoch_loss)
                raceDB.detach_all_hidden_states()
                setup_loss = 1
                wandb.log({"loss": epoch_loss}, step = example_ct)



            #[r.lstm_detach for r in race]
        wandb.log({"epoch_loss": epoch_loss/batch_ct}, step = example_ct)
        acc = validate_model(model,raceDB,criterion, 8, example_ct, epoch_loss, batch_ct)
        print(acc)
        scheduler.step(acc)
    print(losses)
    return model



In [16]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( 'dog_stats_df_FASTTRACK new margins.npy', 'rb')
hidden_size = 64
raceDB = build_dataset(dog_stats_file, hidden_size)

num_features_per_dog=16


100%|██████████| 142750/142750 [02:14<00:00, 1059.31it/s]


number of races = 142750, number of unique dogs = 38030


In [17]:
os.getcwd()
#os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\forms\prediction_inputs_gru.npy', 'rb')
hidden_size = 64
predDB = build_pred_dataset(dog_stats_file, hidden_size)

100%|██████████| 48/48 [00:00<00:00, 1676.16it/s]

number of races = 48, number of unique dogs = 363


In [18]:
# raceDB.create_hidden_states_dict()
# predDB.fill_hidden_states_from_dict(hidden_dict=dataset.hidden_states_dict)
# len(raceDB.hidden_states_dict)
# predDB.to_cuda()

In [19]:
#Testing
def predict_model(model,predDB):
    torch.autograd.set_detect_anomaly(True)
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():

        races_idx = range(0,len(predDB.raceIDs)-1)
        race = predDB.get_race_input(races_idx)
        X = race
        # for i,r in enumerate(race):
        #     print(r.raceid, r.track_name)
        #     #print(i,r.lstm_input())

        output = model(X)
        
        print(output)

        _, predicted = torch.max(output.data, 1)

        for i,r in enumerate(race):
            print(r.raceid, r.track_name, r.dogs[predicted[i].item()])

        print(predicted)



In [20]:
predDB.get_race_input([1,2,3])

In [21]:
# predict_model(model,predDB)

In [22]:
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\LSTM")

In [23]:
os.getcwd()
#os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA\dog_stats_df_FASTTRACK.npy", 'rb')
resultsdf = pickle.load(dog_stats_file)
dog_stats_df = pd.DataFrame(resultsdf)
dog_stats_df = dog_stats_df.sort_values('date')
dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])
dog_stats_df

,dogid,dog_name,raceid,race_grade,date,trackOHE,track_name,dist,stats,place,startprice,margin,bfSP,box
0,214382420,SUPERIOR POWER,498695480,Mixed 6/7 Final,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Healesville,350.0,"[4, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",8.0,1.9,6.87,2.28,4
1128,251075222,IRONDALE VAIRE,500543172,Grade 5,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Dubbo,318.0,"[5, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",4.0,10.0,6.25,NaN,5
1127,120725478,MY LADY MOSEL,500543172,Grade 5,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Dubbo,318.0,"[4, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",6.0,48.7,9.25,NaN,4
1126,120321663,BILLY BOOM BOOM,500543172,Grade 5,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Dubbo,318.0,"[8, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",7.0,12.6,12.25,NaN,8
1125,257638376,TUPELO,500543172,Grade 5,2019-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Dubbo,318.0,"[6, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, ...",8.0,58.0,12.75,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012266,374435883,BIG LEIGH,846606719,Restricted Win,2022-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Mount Gambier,512.0,"[5, 0.059861103060332356, 0.001177721031598112...",8.0,41.0,14.00,NaN,5
1012265,653466830,ZING ZANG PARIS,846606719,Restricted Win,2022-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Mount Gambier,512.0,"[7, 0.059157582438801405, 0.000990221013483566...",7.0,15.0,13.25,NaN,7
1012264,587948676,WHO KNOWS CHLOE,846606719,Restricted Win,2022-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Mount Gambier,512.0,"[6, 0.05941864814027886, 0.0008248952559170212...",4.0,34.0,9.25,NaN,6
1012272,530037960,MYOLA RED BOY,846606720,Restricted Win,2022-12-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Mount Gambier,512.0,"[8, 0.059489962715796776, 0.001022862473588566...",7.0,51.0,3.75,NaN,8


In [24]:
len(dog_stats_df[dog_stats_df['dogid']=='145856635'])

0

In [25]:
weights = (1-(dog_stats_df[dog_stats_df['place']==1]['box'].value_counts(sort=False)/len(dog_stats_df[dog_stats_df['place']==1]))).tolist()
weights = torch.tensor(weights).to(device)
weights

tensor([0.8147, 0.8505, 0.8917, 0.8739, 0.9021, 0.9038, 0.8891, 0.8743],
       device='cuda:0')

In [26]:
(dog_stats_df[dog_stats_df['place']==1]['box'].value_counts(sort=False)/len(dog_stats_df[dog_stats_df['place']==1])).tolist()

[0.1853216783216783,
 0.1495174825174825,
 0.10832867132867133,
 0.12611888111888112,
 0.09788811188811189,
 0.09621678321678322,
 0.11093706293706293,
 0.12567132867132866]

In [27]:
def weighted_mse_loss(input, target, weight):
        return (weight * (input - target) ** 2).sum() / weight.sum()

In [28]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="debug", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)

      model = GRUNet(203,config['hidden_size'])
      # criterion = nn.HuberLoss()
      # criterion = nn.BCEWithLogitsLoss()
      #optimizer = optim.SGD(model.parameters(), lr=0.1)
      #criterion = nn.NLLLoss()
      criterion = nn.CrossEntropyLoss(weight=weights)
      # criterion = nn.SmoothL1Loss(reduction='mean', beta=0.1)

      # optimizer = optim.RMSprop(model.parameters())
      # optimizer = optim.AdamW(model.parameters(), lr=0.001)
      optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',threshold=0.001, patience=100, verbose=True, factor=0.5)
      # optimizer = optim.LBFGS(model.parameters(), lr=0.001)
      model = model.to(device)
      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      train(model, dataset, criterion, optimizer, scheduler, config)
      dataset.create_hidden_states_dict()
      # if sweep:
    #   raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset)

In [29]:
test_race = raceDB.get_race_input([100,101])[0]

In [30]:
len(raceDB.raceIDs)

142750

In [31]:

wandb_config_static = {'hidden_size':hidden_size,'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta':0.1,  'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}

In [32]:
raceDB.detach_all_hidden_states()

In [33]:
sweep_config = {"method": "random"}

metric = {"name": "profit", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": 70000},
    "hidden_size": {"value":16}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [250]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "L1"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00011,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [500,1000,5000]
            # "values": [32, 64, 128, 360, 720]
            # 'values':[4,8,16,32,64,128,360]
        },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config


{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'profit'},
 'parameters': {'batch_size': {'values': [500, 1000, 5000]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [250]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 16},
                'l1_beta': {'value': 0.1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 0.00011},
                'len_data': {'value': 70000},
                'loss': {'values': ['L1']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'profit', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 70000},
  'hidden_size': {'value': 16},
  'epochs': {'values': [250]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['L1']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 0.00011, 'max': 0.001},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [500, 1000, 5000]}}}

In [56]:
(model,dataset) = model_pipeline(raceDB,config=wandb_config_static)

{'hidden_size': 64, 'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
1000
{'hidden_size': 64, 'batch_size': 500, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
GRUNet(
  (gru1): GRUCell(203, 64)
  (gru2): GRUCell(203, 64)
  (gru3): GRUCell(203, 64)
  (gru4): GRUCell(203, 64)
  (gru5): GRUCell(203, 64)
  (gru6): GRUCell(203, 64)
  (gru7): GRUCell(203, 64)
  (gru8): GRUCell(203, 64)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (drop2): Dropout(p=0.3, inplace=False)
  (rl2): ReLU()
  (fc3): Linear(in_features=64, out_features=8, bias=True)
)

Before Loss last 800
final epoch before loss = 183.99655151367188
Af

  0%|          | 0/1230 [00:00<?, ?it/s]

After Loss last 123000


100%|██████████| 1230/1230 [00:02<00:00, 438.34it/s]


accuray: 0.18414634146341463
0.18414634146341463

Before Loss last 800
final epoch before loss = 180.3776092529297
After Loss last 1000

Before Loss last 1800
final epoch before loss = 181.054443359375
After Loss last 2000

Before Loss last 2800
final epoch before loss = 180.13858032226562
After Loss last 3000

Before Loss last 3800
final epoch before loss = 180.43995666503906
After Loss last 4000

Before Loss last 4800
final epoch before loss = 181.9734649658203
After Loss last 5000

Before Loss last 5800
final epoch before loss = 180.9594268798828
After Loss last 6000

Before Loss last 6800
final epoch before loss = 180.80625915527344
After Loss last 7000

Before Loss last 7800
final epoch before loss = 181.05506896972656
After Loss last 8000

Before Loss last 8800
final epoch before loss = 181.02081298828125
After Loss last 9000

Before Loss last 9800
final epoch before loss = 181.07798767089844
After Loss last 10000

Before Loss last 10800
final epoch before loss = 179.427841186523

...

accuray: 0.28447154471544717
0.28447154471544717

Before Loss last 800
final epoch before loss = 167.59048461914062
After Loss last 1000

Before Loss last 1800
final epoch before loss = 165.17417907714844
After Loss last 2000

Before Loss last 2800
final epoch before loss = 165.52374267578125
After Loss last 3000

Before Loss last 3800
final epoch before loss = 160.30795288085938
After Loss last 4000

Before Loss last 4800
final epoch before loss = 167.00537109375
After Loss last 5000

Before Loss last 5800
final epoch before loss = 163.12120056152344
After Loss last 6000

Before Loss last 6800
final epoch before loss = 164.98870849609375
After Loss last 7000

Before Loss last 7800
final epoch before loss = 159.60968017578125
After Loss last 8000

Before Loss last 8800
final epoch before loss = 162.3665313720703
After Loss last 9000

Before Loss last 9800
final epoch before loss = 159.861328125
After Loss last 10000

Before Loss last 10800
final epoch before loss = 158.82008361816406
A

  2%|▏         | 20/1230 [00:00<00:06, 192.13it/s]

After Loss last 123000


100%|██████████| 1230/1230 [00:09<00:00, 129.85it/s]


accuray: 0.2844308943089431
0.2844308943089431

Before Loss last 800
final epoch before loss = 166.47271728515625
After Loss last 1000

Before Loss last 1800
final epoch before loss = 163.29811096191406
After Loss last 2000

Before Loss last 2800
final epoch before loss = 165.17430114746094
After Loss last 3000

Before Loss last 3800
final epoch before loss = 160.4980926513672
After Loss last 4000

Before Loss last 4800
final epoch before loss = 166.69923400878906
After Loss last 5000

Before Loss last 5800
final epoch before loss = 163.92616271972656
After Loss last 6000

Before Loss last 6800
final epoch before loss = 165.075439453125
After Loss last 7000

Before Loss last 7800
final epoch before loss = 161.4325408935547
After Loss last 8000

Before Loss last 8800
final epoch before loss = 162.3880615234375
After Loss last 9000

Before Loss last 9800
final epoch before loss = 159.54443359375
After Loss last 10000

Before Loss last 10800
final epoch before loss = 160.1263427734375
Aft

In [ ]:
pause
dataset.create_hidden_states_dict()

In [ ]:
predDB.fill_hidden_states_from_dict(hidden_dict=dataset.hidden_states_dict)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)